## Homework 1

### Question 1

What's the version of pip in the python:3.13 image

In [ ]:
!docker run -it --entrypoint bash python:3.13

#inside Docker container
!pip --version

Results indicate that pip version is 25.3

## Exercise 2


Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

Reading from the file the paragraph starting with db is important


```
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data
```

According to this paragraph the hostname should that should be used by pgadmin to access the database is **db** and the port is **5432**

## Exercise 3

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?


First I download the data in the Homework1 folder





In [1]:
!cd /workspaces/DataEngineering_DTClub/Homework1/homework

In [2]:
#Download green taxi trips data from Nov 2025
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet

# Download zones data from github
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

--2026-01-25 15:09:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.83, 18.239.38.163, 18.239.38.181, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1164775 (1.1M) [binary/octet-stream]
Saving to: ‘green_tripdata_2025-11.parquet’

green_tripdata_2025 100%[===================>]   1.11M  --.-KB/s    in 0.01s   

2026-01-25 15:09:05 (115 MB/s) - ‘green_tripdata_2025-11.parquet’ saved [1164775/1164775]

--2026-01-25 15:09:05--  https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-productio

The transformation of the data can be found in DataImport.ipynb

After importing the data and looking at its overall structure I ingested the data into postgres with the scripts **ingest_tripsdata_homework** and **ingest_zonesdata_homework**

## Running Postgres and SQL commands

The following SQL commands were run in pgcli

#### Question 3

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?

In [ ]:
SELECT COUNT(*) AS short_trips
FROM trips
WHERE t."trip_distance" <= 1
  AND t."lpep_pickup_datetime" >= '2025-11-01'
  AND t."lpep_pickup_datetime" < '2025-12-01';



#### Question 4


Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).

In [ ]:
SELECT
    DATE(lpep_pickup_datetime) AS pickup_day,
    MAX(trip_distance) AS longest_trip_distance
FROM trips
WHERE trip_distance < 100
GROUP BY pickup_day
ORDER BY longest_trip_distance DESC
LIMIT 1;


#### Question 5


Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

In [ ]:
SELECT
    z."Zone" AS pickup_zone,
    SUM(t."total_amount") AS total_revenue
FROM trips t
JOIN zones z
    ON t."PULocationID" = z."LocationID"
WHERE DATE(t."lpep_pickup_datetime") = '2025-11-18'
GROUP BY z."Zone"
ORDER BY total_revenue DESC
LIMIT 1;

#### Question 6


For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

In [ ]:
SELECT
    dz."Zone" AS dropoff_zone,
    t."tip_amount"
FROM trips t
JOIN zones pz
    ON t."PULocationID" = pz."LocationID"
JOIN zones dz
    ON t."DOLocationID" = dz."LocationID"
WHERE pz."Zone" = 'East Harlem North'
  AND t."lpep_pickup_datetime" >= '2025-11-01'
  AND t."lpep_pickup_datetime" < '2025-12-01'
ORDER BY t."tip_amount" DESC
LIMIT 1;
